In [112]:
#IMPORT LIBRARIES AND REFORMATTED FILES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline

#!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input

import seaborn as sns      #come back and see if need all these!

import openpyxl

import spkit as sp
from spkit.data import load_data

import pywt
from ast import literal_eval
from scipy.signal import butter, filtfilt

import mne
from mne.decoding import CSP

In [196]:
#Load in open source data (https://academic.oup.com/gigascience/article/6/7/gix034/3796323)
left_open = np.genfromtxt('imagery_left.csv', delimiter=',', max_rows=64)
right_open = np.genfromtxt('imagery_right.csv', delimiter=',', max_rows=64)
event_open = np.genfromtxt('imagery_event.csv', delimiter=',', max_rows=64)

indices = np.where(imagery_event == 1)[0] # indices of event starts

fs = 512
n_events = indices.shape[0]
n_ch = left_open.shape[0]
n_samples = left_open.shape[1]

In [198]:
print(left_open)
print(left_open.shape) 
print(right_open.shape)
print(event_open.shape)

print(n_events)
print(n_ch)
print(n_samples)

[[ -28069.75  -28114.75  -28027.   ...  -42884.5   -42741.75  -42729.75]
 [-144500.   -144402.2  -144172.   ... -153728.   -153556.2  -153519.5 ]
 [-168635.   -168614.2  -168545.   ... -206995.   -206875.5  -206862.  ]
 ...
 [  26851.25   26955.25   26885.75 ...  -57658.25  -57749.75  -57981.75]
 [ 199398.8   199600.8   200004.5  ...  181683.8   182446.    182777.  ]
 [-130055.   -130025.5  -129985.5  ... -127611.5  -127576.   -127681.  ]]
(64, 358400)
(64, 358400)
(358400,)
100
64
358400


In [200]:
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    """
    Apply bandpass filter to the inputted data

    Parameters:
    - data : EEG data
    - lowcut : Lowcut frequency in Hz
    - highcut : Highcut frequency in Hz
    - fs : Sampling frequency in Hz

    Returns:
    - y : Filtered data (ch, n)
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data, axis=-1)
    return y

def normalize(data):
    """
    Z-score normalization of the inputted data (mean and std deviation calculated along 1st axis)

    Parameters:
    - data : data to be normalized

    Returns:
    - Normalized data (keeps the dimensions)
    """
    return (data - np.mean(data, axis=1, keepdims=True))/np.std(data, axis=1, keepdims=True)

# Define filter ranges (mu and beta)
mu_low = 7
mu_high = 13
beta_low = 13
beta_high = 30

In [202]:
#FILTER, NORMALIZE, REORGANIZE OPEN SOURCE DATA
left_open = bandpass_filter(left_open, mu_low, beta_high, fs)    #bandpass from 7-30 hz
right_open = bandpass_filter(right_open, mu_low, beta_high, fs)

left_open = normalize(left_open)                          #normalize
right_open = normalize(right_open)

left_open_sliced = np.zeros((n_events, n_ch, 3*fs))     #events, channels, 3 seconds of EEG data
right_open_sliced = np.zeros((n_events, n_ch, 3*fs))

for i, index in enumerate(indices):                       #reorganize based on start of event to 3 seconds following
    left_open_sliced[i] = left_open[:, index:index+3*fs]
    right_open_sliced[i] = right_open[:, index:index+3*fs]

left_open = left_open_sliced
right_open = right_open_sliced

print(left_open.shape)
print(right_open.shape)

imagery_open = np.concatenate((left_open, right_open))     #full set of imagery data
labels_open = np.concatenate((np.ones(n_events), np.zeros(n_events)))   #label = 1 for left, 0 for right

random_indices = np.random.permutation(2*n_events)      #randomize order of data and labels
imagery_open = imagery_open[random_indices]
labels_open = labels_open[random_indices]

indices_to_keep = [12, 49, 47, 20, 30, 57]        #keep only relevant electrodes (c3, c4, cz, p3, p4, pz)
imagery_open = imagery_open[:, indices_to_keep, :]

print(imagery_open.shape)
print(labels_open.shape)
print(labels_open)

(100, 64, 1536)
(100, 64, 1536)
(200, 6, 1536)
(200,)
[0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 0.]


In [204]:
#Save processed open source data and labels
np.save('imagery_open_full.npy', imagery_open)
np.save('labels_open_full.npy', labels_open)

In [206]:
#Load preprocessed open source data and labels
imagery_open = np.load('imagery_open_full.npy')
labels_open = np.load('labels_open_full.npy')

print(imagery_open.shape)
print(labels_open.shape)
print(labels_open)

(200, 6, 1536)
(200,)
[0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 0.]


In [208]:
#Create CNN
model = Sequential()

#adding layers
model.add(Input(shape=(imagery_open.shape[1], 3*fs, 1)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 4, 1534, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 2, 1532, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 1, 766, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 1, 766, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 49024)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │     6,275,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,294,145 (24.01 MB)

 Trainable params: 6,294,145 (24.01 MB)

 Non-trainable params: 0 (0.00 B)

None


In [210]:
# Reshape and split your data
imagery_open_reshaped = imagery_open.reshape(imagery_open.shape[0], imagery_open.shape[1], imagery_open.shape[2], 1) # Add a fourth dim for CNN
print(imagery_open_reshaped.shape)

X_train, X_test, y_train, y_test = train_test_split(imagery_open_reshaped, labels_open, test_size=0.2, random_state=42)

(200, 6, 1536, 1)


In [212]:
# Fit the model to training data
history = model.fit(X_train, y_train, batch_size = 1, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.4094 - loss: 1.0383 - val_accuracy: 0.4062 - val_loss: 0.6998
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.5885 - loss: 0.7133 - val_accuracy: 0.4062 - val_loss: 0.6967
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.6442 - loss: 0.6928 - val_accuracy: 0.4062 - val_loss: 0.8211
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.7542 - loss: 0.4908 - val_accuracy: 0.6250 - val_loss: 0.6967
Epoch 5/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.8330 - loss: 0.3819 - val_accuracy: 0.4062 - val_loss: 0.9167
Epoch 6/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9782 - loss: 0.1387 - val_accuracy: 0.5312 - val_loss: 0.8568
Epoch 7/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9677 - loss: 0.0895 - val_accuracy: 0.4375 - val_loss: 1.5919
Epoch 8/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9895 - loss: 0.0472 - val_accu

In [214]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8042 - loss: 1.4355 
Test Loss: 1.3624969720840454, Test Accuracy: 0.800000011920929
